# Análisis de Egresos 2023 siguiendo los pasos de la "Norma MINSAL 2018-2019. Grupos Relacionados por Diagnóstico Refinados internacionales (GRD IR).

Se analizan los datos del Conjunto Mínimo Básico de Datos (CMBD), obtenidos desde https://www.fonasa.cl/sites/fonasa/datos-abiertos/bases-grd.

- Se considera sólo la actividad de Hospitalización.

- Se excluyen pacientes en GRD’s con más de 365 días de estancia.

- Se excluyen los GRD’s inespecíficos (CDM 99).

- Se excluyen los GRD’s con menos de 10 casos.

- Se excluyen los casos de comportamiento anómalo, pacientes cuyo motivo de alta fue alta voluntaria o fuga.






In [10]:
import pandas as pd
import numpy as np
import math

# Importar librerias
df = pd.read_csv('GRD_PUBLICO_2023.csv', encoding='latin-1', delimiter=';')
pd.set_option('display.max_columns', None)

# Limpieza y arreglo de datos
df['Dias_Estancia'] = (pd.to_datetime(df['FECHAALTA'], format='%d-%m-%Y') - pd.to_datetime(df['FECHA_INGRESO'], format='%d-%m-%Y')).dt.days
df['Dias_Estancia'] = df['Dias_Estancia'].astype(int)
df['IR_29301_COD_GRD'] = df['IR_29301_COD_GRD'].astype(str)

# transformar los valores nan en 0
df['IR_29301_SEVERIDAD'] = df['IR_29301_SEVERIDAD'].fillna(0)

# dejar los valores de IR_29301_COD_GRD sin el .0
df['IR_29301_COD_GRD'] = df['IR_29301_COD_GRD'].str.replace('.0', '', regex=False)

# Eliminar filas donde 'IR_29301_COD_GRD' comienza con '99'
df = df[~df['IR_29301_COD_GRD'].str.startswith('99')]

# Eliminar filas con DIAS_Estancia negativo
df = df[df['Dias_Estancia'] >= 0]

# Eliminar filas con Dias_Estancia mayor a 365
df = df[df['Dias_Estancia'] <= 365]

# Crear un df eliminando los GRD´s con menos de 10 casos
value_counts = df['IR_29301_COD_GRD'].value_counts()
filtered_counts = value_counts[value_counts >= 10]
df_10 = df[df['IR_29301_COD_GRD'].isin(filtered_counts.index)]

# Eliminar filas con TIPOALTA = FUGA DEL PACIENTE o ALTA VOLUNTARIA
df_10 = df_10[~df_10['TIPOALTA'].isin(['FUGA DEL PACIENTE', 'ALTA VOLUNTARIA'])]

# Calcular los puntos de corte para cada GRD
def calcular_puntos_corte(df_10):
  puntos_corte = df_10.groupby('IR_29301_COD_GRD')['Dias_Estancia'].agg(['quantile'])
  puntos_corte['Q1'] = df_10.groupby('IR_29301_COD_GRD')['Dias_Estancia'].quantile(0.25)
  puntos_corte['Q3'] = df_10.groupby('IR_29301_COD_GRD')['Dias_Estancia'].quantile(0.75)
  puntos_corte['RIC'] = puntos_corte['Q3'] - puntos_corte['Q1']
  puntos_corte['Corte_Inferior'] = np.maximum(0, puntos_corte['Q1'] - 1.5 * puntos_corte['RIC'])
  puntos_corte['Corte_Superior'] = puntos_corte['Q3'] + 1.5 * puntos_corte['RIC']
  return puntos_corte

puntos_corte_df = calcular_puntos_corte(df_10)

# Redondear los puntos de corte a números enteros
puntos_corte_df['Corte_Inferior'] = puntos_corte_df['Corte_Inferior'].apply(lambda x: math.ceil(x))
puntos_corte_df['Corte_Superior'] = puntos_corte_df['Corte_Superior'].apply(lambda x: int(x))

# Identificar outliers
def identificar_outliers(row):
  grd = row['IR_29301_COD_GRD']
  estancia = row['Dias_Estancia']

  if grd in puntos_corte_df.index:
    corte_inferior = puntos_corte_df.loc[grd, 'Corte_Inferior']
    corte_superior = puntos_corte_df.loc[grd, 'Corte_Superior']

    if estancia > corte_superior:
      return 'Outlier Superior'
    elif estancia < corte_inferior:
      return 'Outlier Inferior'
    else:
      return 'Normal'
  else:
    return 'Normal'

# Aplicar la función para identificar outliers
df_10['Tipo_Outlier'] = df_10.apply(identificar_outliers, axis=1)

# Crear un DataFrame vacío con las columnas deseadas
df_resultados = pd.DataFrame(columns=['GRD', 'Sev', 'Altas', 'Estancia', 'EM', 'Altas (dep)', 'Est (dep)', 'EM (dep)', 'Outlier Inf', 'Outlier Sup', 'Éxitus', 'Perc 25', 'Perc 75', 'PC inf', 'PC sup'])

# Agrupar por GRD y calcular las estadísticas
for grd in df_10['IR_29301_COD_GRD'].unique():
  df_grd = df_10[df_10['IR_29301_COD_GRD'] == grd]
  IR_29301_SEVERIDAD = df_grd['IR_29301_SEVERIDAD'].iloc[0] if not df_grd['IR_29301_SEVERIDAD'].empty else None
  altas = df_grd.shape[0]
  estancia_total = df_grd['Dias_Estancia'].sum()
  em = estancia_total / altas if altas > 0 else 0
  df_grd_inliers = df_grd[df_grd['Tipo_Outlier'] == 'Normal']
  altas_dep = df_grd_inliers.shape[0]
  estancia_total_dep = df_grd_inliers['Dias_Estancia'].sum()
  em_dep = estancia_total_dep / altas_dep if altas_dep > 0 else 0
  outliers_inf = df_grd[df_grd['Tipo_Outlier'] == 'Outlier Inferior'].shape[0]
  outliers_sup = df_grd[df_grd['Tipo_Outlier'] == 'Outlier Superior'].shape[0]
  perc_25 = df_grd_inliers['Dias_Estancia'].quantile(0.25)
  perc_75 = df_grd_inliers['Dias_Estancia'].quantile(0.75)
  pc_inf = puntos_corte_df.loc[grd, 'Corte_Inferior'] if grd in puntos_corte_df.index else None
  pc_sup = puntos_corte_df.loc[grd, 'Corte_Superior'] if grd in puntos_corte_df.index else None
  exitus = df_grd[df_grd['TIPOALTA'] == 'FALLECIDO'].shape[0]
  df_resultados = pd.concat([df_resultados, pd.DataFrame({'GRD': [grd],
                                                         'Sev': [IR_29301_SEVERIDAD],
                                                         'Altas': [altas],
                                                         'Estancia': [estancia_total],
                                                         'EM': [round(em, 2)],
                                                         'Altas (dep)': [altas_dep],
                                                         'Est (dep)': [estancia_total_dep],
                                                         'EM (dep)': [round(estancia_total_dep / altas_dep, 2) if altas_dep > 0 else 0],
                                                         'Outlier Inf': [outliers_inf],
                                                         'Outlier Sup': [outliers_sup],
                                                         'Éxitus': [exitus],
                                                         'Perc 25': [perc_25],
                                                         'Perc 75': [perc_75],
                                                         'PC inf': [pc_inf],
                                                         'PC sup': [pc_sup]})], ignore_index=True)




print(df_resultados)

<ipython-input-10-b78f6f3c23f8>:6: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,43,47,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,105,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('GRD_PUBLICO_2023.csv', encoding='latin-1', delimiter=';')
<ipython-input-10-b78f6f3c23f8>:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados = pd.concat([df_resultados, pd.DataFrame({'GRD': [grd],


        GRD  Sev Altas Estancia     EM Altas (dep) Est (dep)  EM (dep)  \
0     64161  1.0   167      984   5.89         151       752      4.98   
1     41301  1.0   672     3638   5.41         629      2751      4.37   
2     44161  1.0  3887    15542   4.00        3770     13900      3.69   
3    131301  1.0  7793    15453   1.98        7043      9633      1.37   
4     74131  1.0  1772     7008   3.95        1545      3902      2.53   
..      ...  ...   ...      ...    ...         ...       ...       ...   
910   31133  3.0    11      180  16.36          10       134     13.40   
911     nan  0.0    28      573  20.46          28       573     20.46   
912   41011  1.0    11      373  33.91          10       287     28.70   
913   54111  1.0    17      233  13.71          16       191     11.94   
914  204101  1.0     1       10  10.00           1        10     10.00   

    Outlier Inf Outlier Sup Éxitus  Perc 25  Perc 75 PC inf PC sup  
0             0          16      0     3.0

In [11]:
excel_file_path = 'GRD DESC.xlsx'

# cargar archivo excel con columna que contiene la descripción o nombre del GRD correspondiente
df_grd_desc = pd.read_excel(excel_file_path, sheet_name='Hoja1')

In [12]:
# cambiar el tipo de dato de GRD de df_resultados a numerico

# Convertir columna GRD a numérica
df_resultados['GRD'] = pd.to_numeric(df_resultados['GRD'], errors='coerce')

# Fill NaN
df_resultados['GRD'] = df_resultados['GRD'].fillna(0)

# Imprimir df_resultados
# df_resultados.dtypes

In [13]:
# Agregar una columa DESC (Descripción) a df_resultados que contenga la referencia desde el archivo GRD DESC.xlsx desde la Hoja1.
# Para ver la descripcion en df_resultados comparando los GRD. Dejando la nueva columna al lado derecho de la columna GRD

# Combinar los resultados
df_resultados = pd.merge(df_resultados, df_grd_desc, left_on='GRD', right_on='GRD', how='left')

# Renombrar la columna 'DESC' como 'Descripción GRD'
df_resultados = df_resultados.rename(columns={'DESC': 'Descripción GRD'})

df_resultados

,GRD,Sev,Altas,Estancia,EM,Altas (dep),Est (dep),EM (dep),Outlier Inf,Outlier Sup,Éxitus,Perc 25,Perc 75,PC inf,PC sup,Descripción GRD
0,64161.0,1.0,167,984,5.89,151,752,4.98,0,16,0,3.00,7.00,0,11,MH INFECCIONES GASTROINTESTINALES MAYORES
1,41301.0,1.0,672,3638,5.41,629,2751,4.37,0,43,2,2.00,6.00,0,14,PH PROCEDIMIENTOS MODERADAMENTE COMPLEJOS SOBR...
2,44161.0,1.0,3887,15542,4.00,3770,13900,3.69,0,117,14,2.00,5.00,0,9,MH NEUMONÍA SIMPLE Y TOS FERINA
3,131301.0,1.0,7793,15453,1.98,7043,9633,1.37,0,750,0,1.00,2.00,0,3,PH HISTERECTOMIA VAGINAL Y OTROS PROCEDIMIENTO...
4,74131.0,1.0,1772,7008,3.95,1545,3902,2.53,0,227,9,2.00,3.00,0,7,"MH ENFERMEDADES HEPÁTICAS EXCEPTO NEOPLASIAS, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,31133.0,3.0,11,180,16.36,10,134,13.40,0,1,0,4.00,18.50,0,42,PH PROCEDIMIENTOS SOBRE GLÁNDULAS SALIVARES W/MCC
911,0.0,0.0,28,573,20.46,28,573,20.46,0,0,14,4.50,35.00,0,80,NaN
912,41011.0,1.0,11,373,33.91,10,287,28.70,0,1,2,12.50,39.75,0,80,PH ECMO VENTILACIÓN MECÁNICA PROLONGADA CON TR...
913,54111.0,1.0,17,233,13.71,16,191,11.94,0,1,1,6.75,16.25,0,32,MH ENDOCARDITIS AGUDA Y SUBAGUDA


In [14]:
# Mover la columna 'Descripción GRD' a la segunda posición
cols = list(df_resultados.columns)
cols.insert(1, cols.pop(cols.index('Descripción GRD')))
df_resultados = df_resultados.loc[:, cols]

df_resultados

,GRD,Descripción GRD,Sev,Altas,Estancia,EM,Altas (dep),Est (dep),EM (dep),Outlier Inf,Outlier Sup,Éxitus,Perc 25,Perc 75,PC inf,PC sup
0,64161.0,MH INFECCIONES GASTROINTESTINALES MAYORES,1.0,167,984,5.89,151,752,4.98,0,16,0,3.00,7.00,0,11
1,41301.0,PH PROCEDIMIENTOS MODERADAMENTE COMPLEJOS SOBR...,1.0,672,3638,5.41,629,2751,4.37,0,43,2,2.00,6.00,0,14
2,44161.0,MH NEUMONÍA SIMPLE Y TOS FERINA,1.0,3887,15542,4.00,3770,13900,3.69,0,117,14,2.00,5.00,0,9
3,131301.0,PH HISTERECTOMIA VAGINAL Y OTROS PROCEDIMIENTO...,1.0,7793,15453,1.98,7043,9633,1.37,0,750,0,1.00,2.00,0,3
4,74131.0,"MH ENFERMEDADES HEPÁTICAS EXCEPTO NEOPLASIAS, ...",1.0,1772,7008,3.95,1545,3902,2.53,0,227,9,2.00,3.00,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,31133.0,PH PROCEDIMIENTOS SOBRE GLÁNDULAS SALIVARES W/MCC,3.0,11,180,16.36,10,134,13.40,0,1,0,4.00,18.50,0,42
911,0.0,NaN,0.0,28,573,20.46,28,573,20.46,0,0,14,4.50,35.00,0,80
912,41011.0,PH ECMO VENTILACIÓN MECÁNICA PROLONGADA CON TR...,1.0,11,373,33.91,10,287,28.70,0,1,2,12.50,39.75,0,80
913,54111.0,MH ENDOCARDITIS AGUDA Y SUBAGUDA,1.0,17,233,13.71,16,191,11.94,0,1,1,6.75,16.25,0,32


In [15]:
# Crear un archivo Excel con df_resultados
df_resultados.to_excel('resultados_norma_2023v6.xlsx', index=False)

## Conclusiones

Se obtiene un archivo excel con los datos según la norma 2018-2019.

Esto podría alterar el reusltado final ya que si se agruparan estos casos podrían algunos grupos pasar los 10 casos, lo que agregaría quizas otro GRD.

Pasos futuros: Analizar los datos sin agrupación GRD.

